In [1]:
import numpy
from pandas import read_csv
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

/usr/local/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# First XGBoost model for Pima Indians dataset
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
import pandas as pd

In [4]:
# load data
df = pd.read_csv('data_with_weather_1.csv')

In [5]:
df.dtypes

Unnamed: 0                int64
Hour_of_day               int64
Day_date                 object
date_time                object
pickups                   int64
min                       int64
Row_num                   int64
Date                     object
Time..EST.               object
Temp.                   float64
Visibility              float64
Precip                  float64
Conditions               object
Day_of_week               int64
Month                     int64
Week                      int64
X                       float64
Date_1                   object
one_week_lag_pickups      int64
isholiday                 int64
hour_min                 object
dtype: object

In [6]:
df['hour_min'] = df['hour_min'].astype('category')
df['Conditions'] = df['Conditions'].astype('category')
df['Day_of_week'] = df['Day_of_week'].astype('category')
df['isholiday'] = df['isholiday'].astype('category')


In [7]:
df_cat = df[['hour_min','Conditions','Day_of_week','isholiday']]

In [8]:
df_yvar = df[['pickups']]

In [9]:
df_yvar = df_yvar.values

In [10]:
df_yvar.shape

(8064, 1)

In [11]:
df_yvar = numpy.ravel(df_yvar)

In [12]:
df_yvar.shape

(8064,)

In [13]:
df_numeric = df[['Temp.','Visibility','Precip','one_week_lag_pickups']]

In [14]:
encoded_x = None

In [15]:
df_cat = df_cat.values

In [16]:
df_cat.shape[1]

4

In [17]:
for i in range(0,df_cat.shape[1]):
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(df_cat[:,i])
    feature = feature.reshape(df_cat.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    if encoded_x is None:
        encoded_x = feature
    else:
        encoded_x = numpy.concatenate((encoded_x, feature), axis=1)

In [18]:
encoded_x

array([[ 1.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  1.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  1., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [19]:
encoded_x.shape

(8064, 65)

In [20]:
df_numeric = df_numeric.values

In [21]:
encoded_x = numpy.concatenate((encoded_x, df_numeric), axis=1)

In [22]:
encoded_x.shape

(8064, 69)

In [23]:
df_yvar.shape

(8064,)

In [24]:
X_train = encoded_x[0:7728,:]

In [25]:
X_test = encoded_x[7728:8064,:]

In [26]:
y_train = df_yvar[0:7728]


In [27]:
y_test = df_yvar[7728:8064]

In [28]:
y_test.shape

(336,)

In [29]:
X_train.shape

(7728, 69)

In [ ]:
# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
y_rbf = svr_rbf.fit(X, y).predict(X)
y_lin = svr_lin.fit(X, y).predict(X)
y_poly = svr_poly.fit(X, y).predict(X)

In [30]:
from sklearn.svm import SVR
import matplotlib.pyplot as plt

In [45]:
svr_rbf = SVR(kernel='poly', C=1e3,degree = 2)

In [ ]:
y_rbf = svr_rbf.fit(X_train,y_train).predict(X_test)

In [41]:
y_rbf.shape

(336,)

In [42]:
y_rbf = y_rbf.reshape(-1,1)

In [43]:
df_output = pd.DataFrame(numpy.ravel(y_test),numpy.ravel(y_rbf))

In [44]:
df_output.to_csv("svr_ts_1.csv")

In [30]:
# Let's try XGboost algorithm to see if we can get better results
xgb = XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

In [31]:
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.08, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.75)

In [32]:
from sklearn.metrics import explained_variance_score
predictions = xgb.predict(X_test)
print(explained_variance_score(predictions,y_test))

0.84282638599


In [ ]:
b = self.get_booster()
        fs = b.get_fscore()
        all_features = [fs.get(f, 0.) for f in b.feature_names]
        all_features = np.array(all_features, dtype=np.float32)
        return all_features / all_features.sum()

In [43]:
xgb.booster().get_score(importance_type='weight')

{'f0': 25,
 'f1': 22,
 'f10': 11,
 'f11': 20,
 'f12': 19,
 'f13': 31,
 'f14': 21,
 'f15': 20,
 'f16': 22,
 'f17': 17,
 'f18': 16,
 'f19': 19,
 'f2': 40,
 'f20': 20,
 'f21': 13,
 'f22': 24,
 'f23': 30,
 'f24': 65,
 'f25': 64,
 'f26': 31,
 'f27': 35,
 'f28': 22,
 'f29': 34,
 'f3': 71,
 'f30': 31,
 'f31': 34,
 'f32': 34,
 'f33': 40,
 'f34': 48,
 'f35': 40,
 'f36': 29,
 'f37': 56,
 'f38': 66,
 'f39': 45,
 'f4': 24,
 'f40': 37,
 'f41': 33,
 'f42': 17,
 'f43': 21,
 'f44': 21,
 'f45': 32,
 'f46': 27,
 'f47': 27,
 'f48': 171,
 'f49': 42,
 'f5': 23,
 'f50': 4,
 'f51': 119,
 'f52': 224,
 'f53': 141,
 'f54': 18,
 'f55': 99,
 'f57': 209,
 'f58': 197,
 'f59': 179,
 'f6': 18,
 'f60': 152,
 'f61': 120,
 'f62': 138,
 'f63': 171,
 'f64': 72,
 'f66': 1524,
 'f67': 395,
 'f68': 97,
 'f69': 1681,
 'f7': 23,
 'f8': 21,
 'f9': 30}

In [41]:
model.booster().get_score(importance_type='weight')

In [42]:
fs

0.5

In [33]:
def mape_vectorized_v2(a, b): 
    mask = a <> 0
    return (numpy.fabs(a - b)/a)[mask].mean() * 100

In [34]:
predictions = predictions.reshape(-1,1)

In [35]:
y_test = y_test.reshape(-1,1)
y_test.shape

(336, 1)

In [36]:
print(mape_vectorized_v2(y_test,predictions))

10.5430782326


In [78]:
y_test.shape

(336, 1)

In [79]:
df_output = pd.DataFrame(numpy.ravel(y_test),numpy.ravel(predictions))

In [80]:
df_output

,0
451.795349,445
322.188049,404
304.040466,274
192.094391,184
129.569687,140
119.701752,107
100.999138,124
85.696716,142
95.749969,113
88.953293,86


In [81]:
df_output.to_csv("output_xgboost_ts_4.csv")